In [38]:
import itertools
import re
import json
import random

In [39]:
class CNF:
    def __init__(self, dimacs):
        dimacs_tokens = re.split('\s+', dimacs)
        self.num_of_vars = int(dimacs_tokens[2])
        self.num_of_clauses = int(dimacs_tokens[3])
        clauses_dimacs = [int(x) for x in dimacs_tokens[4:]]
        self.clauses = [list(clause) for is_zero, clause 
                    in itertools.groupby(clauses_dimacs, lambda x: x == 0) 
                    if not is_zero]
        
    def evaluate(self, valuation):
        for clause in self.clauses:
            clause_sat = False
            for literal in clause:
                val_idx = abs(literal) - 1
                if literal > 0 and valuation[val_idx] or literal < 0 and not valuation[val_idx]:
                    clause_sat = True
                    break
            if not clause_sat:
                return False
        return True

In [40]:
class Problem:
    def __init__(self, file_name):
        try:
            with open(file_name, 'r') as f:
                dimacs_list = json.load(f)
                self.cnf_list = [CNF(dimacs) for dimacs in dimacs_list]
                self.num_of_vars = self.cnf_list[0].num_of_vars
        except IOError:
            print(f'Error opening file {file_name}')
            exit(1)

In [41]:
class BFSolver:
    @staticmethod
    def solve(problem):
        all_valuations = itertools.product([True, False], repeat=problem.num_of_vars)
        min_num_of_sat = float('inf')
        best_valuation = None
        iters = 0
        for valuation in all_valuations:
            iters += 1
            num_of_sat = 0
            for formula in problem.cnf_list:
                if formula.evaluate(valuation):
                    num_of_sat += 1
            if num_of_sat < min_num_of_sat:
                min_num_of_sat = num_of_sat
                best_valuation = valuation
        return min_num_of_sat, best_valuation

In [42]:
class GeneticSolver:
    def __init__(self, num_of_generations=50, population_size=20, mutation_probability=0.05):
        self.num_of_generations = num_of_generations
        self.population_size = population_size
        self.mutation_probability = mutation_probability

    def _generate_population(self, problem):
        chromosome_dim = problem.num_of_vars
        return [[random.choice([True, False]) for _ in range(chromosome_dim)] for _ in range(self.population_size)]

    def _fitness(self, problem, chromosome):
        num_of_sat = 0
        for formula in problem.cnf_list:
            if formula.evaluate(chromosome):
                num_of_sat = num_of_sat + 1
        return num_of_sat

    def _selection(self, population, fitness_list):
        return random.choices(population, weights=[1 / (fitness + 1) for fitness in fitness_list], k=1)[0]

    def _crossover(self, parent_1, parent_2):
        bp = random.randrange(len(parent_1))
        child_1 = parent_1[:bp] + parent_2[bp:]
        child_2 = parent_2[:bp] + parent_1[bp:]
        return child_1, child_2

    def _mutate(self, chromosome):
        if random.random() < self.mutation_probability:
            index = random.randrange(len(chromosome))
            chromosome[index] = not chromosome[index]

    def solve(self, problem):
        population = self._generate_population(problem)
        fitness_list = [self._fitness(problem, chromosome) for chromosome in population]
        min_sat = min(fitness_list)
        new_population = [None for _ in range(len(population))]
        for i in range(self.num_of_generations):
            for j in range(self.population_size // 2):
                parent_1 = self._selection(population, fitness_list)
                parent_2 = self._selection(population, fitness_list)
                child_1, child_2 = self._crossover(parent_1, parent_2)
                self._mutate(child_1)
                self._mutate(child_2)
                new_population[2 * j] = child_1
                new_population[2 * j + 1] = child_2
            population = new_population
            fitness_list = [self._fitness(problem, chromosome) for chromosome in population]
            min_sat = min(fitness_list + [min_sat])
            print(f'generation[{i}] :  {min_sat}')
        return min_sat

In [43]:
p = Problem('problem_instances/1.json')
bf_solver = BFSolver()
print(BFSolver.solve(p))
ga_solver = GeneticSolver()
print(ga_solver.solve(p))

(30, (True, True, True, False, False, False, False, False, True, True))
generation[0] :  33
generation[1] :  33
generation[2] :  33
generation[3] :  33
generation[4] :  33
generation[5] :  33
generation[6] :  33
generation[7] :  33
generation[8] :  33
generation[9] :  33
generation[10] :  33
generation[11] :  33
generation[12] :  33
generation[13] :  33
generation[14] :  33
generation[15] :  33
generation[16] :  33
generation[17] :  33
generation[18] :  33
generation[19] :  33
generation[20] :  33
generation[21] :  33
generation[22] :  33
generation[23] :  33
generation[24] :  33
generation[25] :  33
generation[26] :  33
generation[27] :  33
generation[28] :  33
generation[29] :  33
generation[30] :  33
generation[31] :  32
generation[32] :  32
generation[33] :  32
generation[34] :  32
generation[35] :  32
generation[36] :  32
generation[37] :  32
generation[38] :  32
generation[39] :  31
generation[40] :  31
generation[41] :  31
generation[42] :  31
generation[43] :  31
generation[44]